# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#! export
from fastcore.utils import *
from typing import List, Dict, Optional
from httpx import get as xget, post as xpost
import json
from IPython.display import Markdown, Image, display
from claudette import *
from pathlib import Path
from functools import wraps, lru_cache
from pathlib import Path
from hashlib import sha256
import s2sphere
import folium
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def display_s2_cells_on_map(lat, lng, levels=[10, 13, 15, 20], zoom_start=15):
    """Display S2 cells at different levels on an interactive map"""
    # Create a map centered at the specified location
    m = folium.Map(location=[lat, lng], zoom_start=zoom_start, 
                  tiles='OpenStreetMap')
    
    # Add a marker for the target point
    folium.Marker(
        [lat, lng], 
        popup=f"Target Point: {lat}, {lng}",
        icon=folium.Icon(color='black', icon='crosshairs', prefix='fa')
    ).add_to(m)
    
    # Colors for different levels
    colors = {10: 'red', 13: 'blue', 15: 'green', 20: 'purple', 22: 'orange'}
    
    # Add cells at different levels
    for level in levels:
        cell_id = s2sphere.CellId.from_lat_lng(
            s2sphere.LatLng.from_degrees(lat, lng)).parent(level)
        cell = s2sphere.Cell(cell_id)
        
        # Get vertices of the cell
        vertices = []
        for i in range(4):
            vertex = cell.get_vertex(i)
            ll = s2sphere.LatLng.from_point(vertex)
            vertices.append([ll.lat().degrees, ll.lng().degrees])
        # Close the polygon
        vertices.append(vertices[0])
        
        # Add to map with different colors by level
        folium.Polygon(
            locations=vertices,
            popup=f"S2 Cell Level {level}<br>ID: {cell_id.id()}<br>Token: {cell_id.to_token()}",
            color=colors[level],
            weight=2,
            fill=True,
            fill_opacity=0.2
        ).add_to(m)
    
    return m

In [ ]:
# University of Notre Dame coordinates (Main Building/Golden Dome)
# notre_dame_lat, notre_dame_lng = 41.7056, -86.2353
notre_dame_lat,notre_dame_lng = 41.704935,-86.234763

In [ ]:
# Create map with S2 cells at different levels
notre_dame_map = display_s2_cells_on_map(
    notre_dame_lat, notre_dame_lng, 
    levels=[10, 13, 15, 20, 22],  # Added level 22 for more detail
    zoom_start=16
)

In [ ]:
notre_dame_map


In [ ]:
def print_s2_cell_info(lat, lng, levels=range(10, 23)):
    """Print information about S2 cell sizes at different levels"""
    print(f"S2 Cell Information for location: {lat}, {lng}")
    print("-" * 60)
    print(f"{'Level':<6} {'Approx Size':<15} {'Cell ID':<20} {'Token':<12}")
    print("-" * 60)
    
    for level in levels:
        cell_id = s2sphere.CellId.from_lat_lng(
            s2sphere.LatLng.from_degrees(lat, lng)).parent(level)
        
        # Calculate approximate size
        cell = s2sphere.Cell(cell_id)
        vertex1 = cell.get_vertex(0)
        vertex2 = cell.get_vertex(1)
        ll1 = s2sphere.LatLng.from_point(vertex1)
        ll2 = s2sphere.LatLng.from_point(vertex2)
        
        # Rough approximation of cell size in meters
        # This is approximate and works best near the equator
        earth_radius = 6371000  # meters
        dlat = abs(ll1.lat().degrees - ll2.lat().degrees) * np.pi/180
        dlng = abs(ll1.lng().degrees - ll2.lng().degrees) * np.pi/180
        
        # Rough calculation of distance
        a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(ll1.lat().radians) * np.cos(ll2.lat().radians) * np.sin(dlng/2) * np.sin(dlng/2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        distance = earth_radius * c
        
        size_desc = f"~{distance:.1f} m"
        
        # Print the info
        print(f"{level:<6} {size_desc:<15} {cell_id.id():<20} {cell_id.to_token():<12}")
    
    return None

In [ ]:
# Print S2 cell information for Notre Dame
print_s2_cell_info(notre_dame_lat, notre_dame_lng, levels=range(15, 23))


S2 Cell Information for location: 41.7056, -86.2353
------------------------------------------------------------
Level  Approx Size     Cell ID              Token       
------------------------------------------------------------
15     ~276.6 m        9806256792143921152  8816d298c   
16     ~138.3 m        9806256791338614784  8816d2989   
17     ~69.1 m         9806256791405723648  8816d29894  
18     ~34.6 m         9806256791456055296  8816d29897  
19     ~17.3 m         9806256791451860992  8816d29896c 
20     ~8.6 m          9806256791448715264  8816d298969 
21     ~4.3 m          9806256791448977408  8816d2989694
22     ~2.2 m          9806256791449042944  8816d2989695


In [ ]:
#| export
def foo(): pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()